In [ ]:
# 1. GEREKLİ KÜTÜPHANE KURULUMLARI VE ORTAM AYARLARI

# Bu adımda RAG mimarisi için gerekli olan tüm kütüphaneler kurulmaktadır.
!pip install google-genai langchain langchain-google-genai langchain-community faiss-cpu opendatasets pandas gradio sentence-transformers -q

import os
from google.colab import userdata
import glob

# RAG Bileşenleri Import'ları
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import Document
import gradio as gr

# Global Değişken Tanımları
KAGGLE_DATASET_URL = "https://www.kaggle.com/datasets/cuddlefish/fairy-tales"
TEXT_FILE_PATH = "initial_placeholder.txt"
FAISS_DB_PATH = "faiss_index_fairy_tales"

# --- API Anahtarlarını Colab Secrets'tan Yükleme ---
print("Colab Secrets'tan API Anahtarları Yükleniyor...")

# GEMINI API Anahtarını Yükleme (LLM için)
GEMINI_API_KEY_VALUE = None
if not os.getenv("GEMINI_API_KEY"):
    try:
        GEMINI_API_KEY_VALUE = userdata.get('GEMINI_API_KEY')
        os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY_VALUE
        print("-> Gemini API Key (LLM için) yüklendi.")
    except Exception:
        print("HATA: 'GEMINI_API_KEY' Colab Secrets'ta bulunamadı.")
else:
    GEMINI_API_KEY_VALUE = os.getenv("GEMINI_API_KEY")
    print("-> Gemini API Key (LLM için) ortam değişkeninden alındı.")

# KAGGLE API Kimlik Bilgilerini Yükleme
if not os.getenv("KAGGLE_USERNAME") or not os.getenv("KAGGLE_KEY"):
    try:
        os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
        os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
        print("-> Kaggle API Kimlik Bilgileri yüklendi.")
    except Exception:
        print("UYARI: Kaggle kimlik bilgileri Secrets'ta bulunamadı.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.
Colab Secrets'tan API Anahtarları Yükleniyor...
-> Gemini API Key (LLM için) yüklendi.
-> Kaggle API Kimlik Bilgiler

In [ ]:
# 2. VERİ SETİ EDİNİMİ VE METİN YOLU DÜZELTMESİ

# Veri setini Kaggle CLI ile indirip unzip ediyoruz.
!kaggle datasets download -d cuddlefish/fairy-tales --unzip
print("--- İndirme Tamamlandı ---")

# --- İndirilen Dosyanın Yolunu Otomatik Bulma ve Güncelleme ---

def find_and_update_path():
    data_file_path = None

    # Doğrudan /content (ana) dizinindeki ve alt klasörlerdeki tüm .txt dosyalarını ara
    txt_files = glob.glob('*.txt')
    for name in os.listdir('.'):
        if os.path.isdir(name) and ('fairy' in name.lower() or 'tales' in name.lower()):
            txt_files.extend(glob.glob(os.path.join(name, '*.txt')))

    if txt_files:
        # En büyük dosyayı seç
        txt_files.sort(key=lambda x: os.path.getsize(x), reverse=True)
        data_file_path = txt_files[0]

        # TEXT_FILE_PATH değişkenini güncelle
        globals()['TEXT_FILE_PATH'] = data_file_path

        print("\n--- TEXT_FILE_PATH BAŞARIYLA GÜNCELLENDİ ---")
        print("Yeni yol:", globals()['TEXT_FILE_PATH'])
        return True
    else:
        print("HATA: Ana dizinde veya alt klasörlerde .txt uzantılı dosya bulunamadı.")
        return False

# Fonksiyonu Çalıştırma
find_and_update_path()

Dataset URL: https://www.kaggle.com/datasets/cuddlefish/fairy-tales
License(s): CC0-1.0
  0% 0.00/4.44M [00:00<?, ?B/s]
100% 4.44M/4.44M [00:00<00:00, 301MB/s]
--- İndirme Tamamlandı ---

--- TEXT_FILE_PATH BAŞARIYLA GÜNCELLENDİ ---
Yeni yol: merged_clean.txt


True

In [ ]:
# 3. VERİ PARÇALAMA (CHUNKING) VE HIZ MODU LİMİTİ

# KRİTİK AYAR: Bu değişkeni False yaparak hızlı teslimat için 1000 parça limiti etkinleştirilir.
FULL_RUN_MODE = False

def load_data():
    """Tüm veri setini okur, parçalar ve FULL_RUN_MODE'a göre limitler."""
    mode_text = "Tüm Koleksiyon (Yavaş)" if FULL_RUN_MODE else "Limitli Koleksiyon (Hızlı)"
    print(f"\n--- Veri Seti Hazırlanıyor ({mode_text}) ---")

    # Metin dosyasını okuma
    try:
        file_path = globals().get('TEXT_FILE_PATH')
        with open(file_path, 'r', encoding='utf-8') as f:
            full_text = f.read()
    except FileNotFoundError:
        print(f"Hata: Metin dosyası bulunamadı: {globals().get('TEXT_FILE_PATH')}")
        return []

    # Basit RecursiveCharacterTextSplitter ayarları
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len
    )

    all_texts = text_splitter.split_text(full_text)
    texts = all_texts

    if not FULL_RUN_MODE:
        MAX_DOCUMENTS = 1000
        texts = all_texts[:MAX_DOCUMENTS]
        print(f"UYARI: Proje hızlandırması için parça sayısı {MAX_DOCUMENTS} ile sınırlandırıldı.")

    documents = [Document(page_content=chunk.strip()) for chunk in texts]

    print(f"Toplam {len(documents)} adet parçalanmış doküman oluşturuldu.")
    return documents

# Fonksiyonu Çalıştırma
documents = load_data()


--- Veri Seti Hazırlanıyor (Limitli Koleksiyon (Hızlı)) ---
UYARI: Proje hızlandırması için parça sayısı 1000 ile sınırlandırıldı.
Toplam 1000 adet parçalanmış doküman oluşturuldu.


In [ ]:
# 4. EMBEDDING VE FAISS İNDEKS OLUŞTURMA (T4 GPU OPTİMİZASYONU)

import shutil # Eski indeksi silmek için

def create_vector_store(documents):
    """Dokümanları embed eder ve FAISS Vektör Veri Tabanı oluşturur/yükler."""
    print("\n--- FAISS Vektör Veri Tabanı Hazırlanıyor ---")

    # Embedding Modeli: all-mpnet-base-v2
    EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"
    print(f"Embedding modeli olarak HuggingFace '{EMBEDDING_MODEL_NAME}' kullanılıyor...")

    # KRİTİK AYAR: Modeli zorla T4 GPU'ya (cuda) yükleme.
    embedding_model = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cuda'}
    )

    # Eski FAISS indeksini silme
    if os.path.exists(FAISS_DB_PATH):
        print(f"UYARI: Yeni veri boyutu kullanıldığı için eski FAISS indeksi ({FAISS_DB_PATH}) SİLİNİYOR.")
        shutil.rmtree(FAISS_DB_PATH)


    print("FAISS indeksi oluşturuluyor ve kaydediliyor...")
    if not documents:
        print("Hata: İşlenecek doküman bulunmadığı için indeks oluşturulamadı.")
        return None

    # Dokümanları embed et ve FAISS'e ekle
    vector_store = FAISS.from_documents(documents, embedding_model)

    # İndeksi kaydet
    vector_store.save_local(FAISS_DB_PATH)
    print(f"FAISS indeksi {FAISS_DB_PATH} konumuna başarıyla kaydedildi.")

    return vector_store

# Fonksiyonu Çalıştırma
vector_store = create_vector_store(documents)


--- FAISS Vektör Veri Tabanı Hazırlanıyor ---
Embedding modeli olarak HuggingFace 'all-mpnet-base-v2' kullanılıyor...


/tmp/ipython-input-1652042957.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS indeksi oluşturuluyor ve kaydediliyor...
FAISS indeksi faiss_index_fairy_tales konumuna başarıyla kaydedildi.


In [ ]:
# 5. RAG ZİNCİRİ KURULUMU VE AKADEMİK PROMPT MÜHENDİSLİĞİ

def setup_rag_chain(vector_store):
    """RAG zincirini (Chain) ayarlar."""
    print("\n--- RAG Zinciri Kuruluyor (Gemini 2.0 Flash) ---")

    if not vector_store or not globals().get('GEMINI_API_KEY_VALUE'):
        print("Hata: Vektör veritabanı veya Gemini API Anahtarı mevcut değil. Zincir oluşturulamadı.")
        return None

    llm = GoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=globals().get('GEMINI_API_KEY_VALUE')
    )

    # Retriever: Arama derinliğini k=8'e çıkarıyoruz.
    retriever = vector_store.as_retriever(search_kwargs={"k": 8})

    # PROMPT GÜNCELLEMESİ: AKADEMİK TON VE SIKI RAG KURALI
    template = """You are an AI research assistant specializing in literary analysis of fairy tales.
    Your task is to answer the user's question **STRICTLY** using the provided context.
    If the answer is not present in the context, politely respond with: 'I am sorry, I could not find this information in the fairy tales available to me.'
    Formulate your answer using an **objective and academic tone** suitable for a literary scholar. Avoid flowery or overly creative language.

    Context: {context}

    Question: {question}

    Answer:"""

    custom_rag_prompt = PromptTemplate.from_template(template)

    # RetrievalQA Zinciri
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=False, # DEBUG kapalı.
        chain_type_kwargs={"prompt": custom_rag_prompt}
    )

    print("RAG zinciri başarıyla kuruldu. Akademik ton (k=8) etkin.")
    return qa_chain

def answer_question(qa_chain, question):
    """RAG zincirini kullanarak bir soruyu cevaplar."""
    if not qa_chain:
        return "Chatbot failed to initialize. Please check previous steps."

    try:
        result = qa_chain.invoke({"query": question})
        answer = result['result']
        return answer

    except Exception as e:
        error_message = f"Error during Gemini/LLM call: {e}"
        return "I am sorry, a problem occurred. Please try again later."

# Fonksiyonu Çalıştırma
rag_chain = setup_rag_chain(vector_store)


--- RAG Zinciri Kuruluyor (Gemini 2.0 Flash) ---
RAG zinciri başarıyla kuruldu. Akademik ton (k=8) etkin.


In [ ]:
# 6. CHATBOT ARAYÜZÜNÜ BAŞLATMA (GRADIO)

def launch_gradio_app(qa_chain):
    """Gradio arayüzünü başlatır."""

    if qa_chain is None:
        def dummy_predict(message, history):
            return "Chatbot failed to initialize. Please check previous steps."
        predict_func = dummy_predict
        title = "Masal Chatbotu (Hata: Başlatılamadı)"
    else:
        def predict(message, history):
            return answer_question(qa_chain, message)

        predict_func = predict
        title = "Tale Me A Story - RAG Tabanlı Masal Chatbotu"

    print("\n--- Gradio Arayüzü Başlatılıyor ---")

    gr.ChatInterface(
        fn=predict_func,
        title=title,
        description="Akademik bir tonda yanıt veren RAG destekli masal analiz asistanı. Lütfen sorlarınızı İngilizce sorun."
    ).launch(share=True)

# Fonksiyonu Çalıştırma
launch_gradio_app(rag_chain)


--- Gradio Arayüzü Başlatılıyor ---


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8b1009007dc18bc0e7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
